In [109]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
from sklearn.metrics import mean_squared_error
#matplotlib.rcParams['text.color'] = 'G'
df_tr = pd.read_excel("Res_1_сглаживание рядов_TR.xlsx")
df = pd.read_excel("Res_1_сглаживание рядов_Reg.xlsx")
df = pd.concat([df, df_tr])


df = df[df['create_date']<'2023-04-01']



y = df.set_index(["create_date"])
y=y[y['Region'] != 'Горно-Алтайск']
y=y[['Region', 'fin']]
y.columns = ['Region', 'nps']
y

,Region,nps
create_date,,
2020-01-01,Архангельск,0.213656
2020-02-01,Архангельск,0.165085
2020-03-01,Архангельск,0.202864
2020-04-01,Архангельск,0.213432
2020-05-01,Архангельск,0.073345
...,...,...
2022-11-01,TR,0.298490
2022-12-01,TR,0.301934
2023-01-01,TR,0.246672


In [110]:
#подбираем параметры для модели
p = d = q = range(0, 2)
PDQ = list(itertools.product(p, d, q))
SEASONAL_PDQ = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print('Examples of parameter for SARIMA...')
print('SARIMAX: {} x {}'.format(PDQ[1], SEASONAL_PDQ[1]))
print('SARIMAX: {} x {}'.format(PDQ[1], SEASONAL_PDQ[2]))
print('SARIMAX: {} x {}'.format(PDQ[2], SEASONAL_PDQ[3]))
print('SARIMAX: {} x {}'.format(PDQ[2], SEASONAL_PDQ[4]))


Examples of parameter for SARIMA...
SARIMAX: (0, 0, 1) x (0, 0, 1, 12)
SARIMAX: (0, 0, 1) x (0, 1, 0, 12)
SARIMAX: (0, 1, 0) x (0, 1, 1, 12)
SARIMAX: (0, 1, 0) x (1, 0, 0, 12)


In [111]:
def reg_forcust(y, region):
    #запускаем перебеор
    ''' for param in PDQ:
        for param_seasonal in SEASONAL_PDQ:
            try:
                mod = sm.tsa.statespace.SARIMAX(y,order=param,seasonal_order=param_seasonal,enforce_stationarity=False,enforce_invertibility=False)
                results = mod.fit()
                print('ARIMA{}x{}12 - AIC:{}'.format(param,param_seasonal,results.aic))
            except: 
                continue
    # распечатываем саммари'''
    mod = sm.tsa.statespace.SARIMAX(y,
                                    order=(1, 1, 1),
                                    seasonal_order=(1, 1, 1,12 ),
                                    #enforce_stationarity=False,
                                    enforce_invertibility=False)
    results = mod.fit()
    #print(results.summary().tables[1])

    pred = results.get_prediction(start=pd.to_datetime('2022-03-01'), dynamic=False)
    #pd.DataFrame(pred.predicted_mean) 
    y_forecasted = pred.predicted_mean
    y_truth = y['2022-03-01':]
    #print(mean_squared_error(y_truth.nps , y_forecasted))
    #прогноз
    pred_uc = results.get_forecast(steps=12)
    forecast = pred_uc.predicted_mean.head(12)
    forecast = pd.DataFrame(forecast)
    forecast['region'] = region
    return forecast

In [112]:
total_df = pd.DataFrame()
for region in y['Region'].unique().tolist():
    #print(region)
    tmp_df = y[y['Region'] == region]
    tmp_df.drop('Region', axis= 1 , inplace= True )
    tmp_df.head()
    region_forecast_df = reg_forcust(tmp_df, region)
    total_df = pd.concat([total_df, region_forecast_df])
    
print(total_df)

            predicted_mean       region
2023-04-01        0.249077  Архангельск
2023-05-01        0.181266  Архангельск
2023-06-01        0.075373  Архангельск
2023-07-01        0.050285  Архангельск
2023-08-01        0.006443  Архангельск
...                    ...          ...
2023-11-01        0.312591           TR
2023-12-01        0.314338           TR
2024-01-01        0.249395           TR
2024-02-01        0.236015           TR
2024-03-01        0.262115           TR

[744 rows x 2 columns]


In [113]:
total_df.columns = ['nps', 'Region']
total_df = total_df[['Region', 'nps']]
total_df

,Region,nps
2023-04-01,Архангельск,0.249077
2023-05-01,Архангельск,0.181266
2023-06-01,Архангельск,0.075373
2023-07-01,Архангельск,0.050285
2023-08-01,Архангельск,0.006443
...,...,...
2023-11-01,TR,0.312591
2023-12-01,TR,0.314338
2024-01-01,TR,0.249395
2024-02-01,TR,0.236015


In [114]:
y

,Region,nps
create_date,,
2020-01-01,Архангельск,0.213656
2020-02-01,Архангельск,0.165085
2020-03-01,Архангельск,0.202864
2020-04-01,Архангельск,0.213432
2020-05-01,Архангельск,0.073345
...,...,...
2022-11-01,TR,0.298490
2022-12-01,TR,0.301934
2023-01-01,TR,0.246672


In [115]:
total = pd.concat([y, total_df])
total.reset_index(inplace = True)
total.columns = ['create_date', 'Region', 'nps_result']
total = total.sort_values(by = ['Region', 'create_date'], ascending=True)
total.reset_index(drop= True , inplace= True )
total

,create_date,Region,nps_result
0,2020-01-01,TR,0.286909
1,2020-02-01,TR,0.270087
2,2020-03-01,TR,0.271270
3,2020-04-01,TR,0.285598
4,2020-05-01,TR,0.252262
...,...,...,...
3157,2023-11-01,Ярославль,0.491513
3158,2023-12-01,Ярославль,0.487754
3159,2024-01-01,Ярославль,0.378421
3160,2024-02-01,Ярославль,0.449736


In [116]:
df.reset_index(drop= True , inplace= True )
df

,Unnamed: 0,create_date,Region,nps,result,nps_roll,fin
0,0,2020-01-01,Архангельск,0.213656,NaN,NaN,0.213656
1,1,2020-02-01,Архангельск,0.165085,NaN,NaN,0.165085
2,2,2020-03-01,Архангельск,0.202864,NaN,NaN,0.202864
3,3,2020-04-01,Архангельск,0.272124,0.272124,0.213432,0.213432
4,4,2020-05-01,Архангельск,0.073345,NaN,0.178355,0.073345
...,...,...,...,...,...,...,...
2418,34,2022-11-01,TR,0.298490,NaN,0.245897,0.298490
2419,35,2022-12-01,TR,0.301934,NaN,0.273695,0.301934
2420,36,2023-01-01,TR,0.246672,NaN,0.277354,0.246672
2421,37,2023-02-01,TR,0.232621,NaN,0.269929,0.232621


In [117]:
df_pivot = pd.merge(total, df, how = "left", left_on=['create_date', 'Region'], right_on = ['create_date','Region'])
df_pivot = df_pivot[['create_date', 'Region', 'nps_result', 'nps', 'result', 'nps_roll', 'fin']]
df_pivot.columns = ['create_date',	'Region',	'nps_result', 'Выгрузка NPS MI', 'Метка выброса', 'Скользящее', 'NPS_с учетом скользящего']
df_pivot

,create_date,Region,nps_result,Выгрузка NPS MI,Метка выброса,Скользящее,NPS_с учетом скользящего
0,2020-01-01,TR,0.286909,0.286909,NaN,NaN,0.286909
1,2020-02-01,TR,0.270087,0.270087,NaN,NaN,0.270087
2,2020-03-01,TR,0.271270,0.271270,NaN,NaN,0.271270
3,2020-04-01,TR,0.285598,0.314125,0.314125,0.285598,0.285598
4,2020-05-01,TR,0.252262,0.252262,NaN,0.276936,0.252262
...,...,...,...,...,...,...,...
3157,2023-11-01,Ярославль,0.491513,NaN,NaN,NaN,NaN
3158,2023-12-01,Ярославль,0.487754,NaN,NaN,NaN,NaN
3159,2024-01-01,Ярославль,0.378421,NaN,NaN,NaN,NaN
3160,2024-02-01,Ярославль,0.449736,NaN,NaN,NaN,NaN


In [118]:
#total.to_excel('forecast_region.xlsx')
df_pivot.to_excel('forecast_region.xlsx')